# Create list of patients with correct MRI sequences

### Imports

In [1]:
import os
import sys
sys.path.append(r"/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/Brain_Mets_Classification")

from tqdm import tqdm
from datetime import datetime
import shutil
import pandas as pd

import brain_mets_classification.config as config
import brain_mets_classification.custom_funcs as funcs

In [ ]:
path_to_folder = f"{config.path}/originalPatientFiles"

### Copy files of patients witht the correct sequences into new directory and create a .csv file with the IDs of said patients

In [ ]:
pathToCleanMRIList = funcs.createNewPreprocessingStepFolder(step=0)

patientFolder = os.listdir(path_to_folder)